# Building-to-Service Travel Time Matrix with Intermodal Graph
This notebook demonstrates how to compute a time-based adjacency matrix between two GeoDataFrames
(e.g., buildings and services) using a multimodal transport graph.

The method utilizes the `IduEdu` library to:
- Construct a multimodal graph (e.g., walk + public transport)
- Calculate travel time-based adjacency matrix from one GeoDataFrame to another

This matrix can be used in `ObjectNat` for further service provision analysis.

In [ ]:
# %%
# Install required packages (uncomment if needed)
# !pip install iduedu pyarrow

In [ ]:
# Import necessary libraries
from iduedu import get_intermodal_graph, get_adj_matrix_gdf_to_gdf
import geopandas as gpd
import pandas as pd
from shapely.ops import unary_union

## 1. Load Input Geospatial Data
Load the GeoDataFrames of buildings (origins) and services (destinations).


In [ ]:
# Read building and service datasets
buildings = gpd.read_parquet('examples_data/buildings.parquet')
services = gpd.read_parquet('examples_data/services.parquet')

## 2. Create Coverage Polygon for Graph Download
Compute a polygon that encompasses both datasets to define the spatial extent for graph download.
This is done by computing a convex hull over all geometries and buffering it slightly.


In [ ]:
polygon = unary_union(
    buildings.to_crs(4326).geometry.to_list() + services.to_crs(4326).geometry.to_list()
).convex_hull.buffer(0.001)

## 3. Download and Clip Intermodal Graph
Download the intermodal (multi-modal) network graph using the defined polygon.
This includes walking paths and public transport networks.

In [ ]:
# Load multimodal graph clipped to polygon
G_intermodal = get_intermodal_graph(territory=polygon, clip_by_territory=True)

## 4. Compute Adjacency Matrix (Travel Time)
Calculate a travel-time-based adjacency matrix from buildings to services.

Parameters:
- `weight`: edge attribute used for cost (e.g., "time_min")
- `threshold`: maximum allowed travel time (in minutes)

In [ ]:
# Compute travel time matrix (in minutes)
matrix: pd.DataFrame = get_adj_matrix_gdf_to_gdf(
    gdf_from=buildings,
    gdf_to=services,
    nx_graph=G_intermodal,
    weight="time_min",
    threshold=45
)

## 5. Save Adjacency Matrix
Export the result for further processing, e.g., with `ObjectNat`'s service provision tools.

In [ ]:
# Save matrix to Parquet format
matrix.to_parquet('examples_data/matrix_time.parquet')